In [ ]:
%load_ext autoreload
%autoreload 2

import yaml

import torch
import pandas as pd
from IPython.core.display import HTML
from IPython.display import display
from torch.nn.functional import normalize

from oml.utils.visualisation import RetrievalVisualizer
from oml.lightning.entrypoints.validate import main
from oml.functional.metrics import calc_retrieval_metrics

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline


In [ ]:
cfg = """

dataset_root: /nydl/data/DeepFashion_InShop/
dataframe_name: df_fixed_val.csv
logs_root: logs
im_size: 304
bs_val: 96
num_workers: 20

model:
  name: vit
  args:
    arch: vits8
    normalise_features: True
    use_multi_scale: False
    weights: /nydl/logs/cur/ml/deepfashion/2022-05-18_12-09-14_deepfashion/checkpoints/best.ckpt
    strict_load: True

gpus:
  - 0
  
"""
        
clb_metric, ret_dict = main(yaml.load(cfg, Loader=yaml.Loader))


In [ ]:
vis = RetrievalVisualizer.from_embeddings_metric(clb_metric.metric)


In [ ]:
n_query = clb_metric.metric.distance_matrix.shape[0]

for i in range(min(10, n_query)):
    vis.visualise(query_idx=1000 + i, top_k=1, skip_no_errors=True)
